In [6]:
# To ignore unimporant system warnings
import warnings
warnings.filterwarnings("ignore")

# We will use Pandas, Numpy, and Matplotlib which is a package for visualization with Python
import pandas as pd
#pd.set_option('display.max_rows',1000)
import numpy as np
import geopandas as gpd
from datetime import datetime
from geodatasets import get_path

# This is a library for accessing and parsing data through URLs
import urllib.request, json 
import urllib.parse
from urllib.parse import urlencode

# Using folium to create a map
import folium
from folium import plugins
import matplotlib.pyplot as plt
import seaborn as sns # visualization styling package
import branca  # For color gradient
%matplotlib inline 

In [18]:
import urllib.request
from urllib.parse import urlencode
import json
import pandas as pd

def fetch_bus_data(route_id=None, date_start=None, date_end=None, borough=None, limit=1000):
    # Define API endpoint and base query
    BASE_API = "https://data.ny.gov/resource/58t6-89vi.json?"
    
    # Initialize the query dictionary
    query_speeds = {
        '$select': 'route_id, timepoint_stop_name,stop_order, direction,AVG(timepoint_stop_latitude) as timepoint_stop_latitude_avg, AVG(timepoint_stop_longitude) as timepoint_stop_longitude_avg',
        '$group': 'route_id, timepoint_stop_name, stop_order, direction',
        '$limit': limit
    }
    
    # Build WHERE clause conditions
    where_conditions = []
    if route_id:
        where_conditions.append(f'route_id="{route_id}"')
    if borough:
        where_conditions.append(f'borough="{borough}"')
    if date_start:
        where_conditions.append(f'timestamp >= "{date_start}T00:00:00"')
    if date_end:
        where_conditions.append(f'timestamp <= "{date_end}T23:59:59"')
    
    # Combine WHERE conditions if any exist
    if where_conditions:
        query_speeds['$where'] = ' AND '.join(where_conditions)
    
    # Construct and execute the query
    url_speeds = BASE_API + urlencode(query_speeds)
    response_speeds = urllib.request.urlopen(url_speeds)
    data_speeds = json.loads(response_speeds.read().decode())
    
    # Convert to DataFrame
    df_speeds = pd.DataFrame(data_speeds)
    return df_speeds



In [19]:
# Example usage
df = fetch_bus_data(route_id="B1", limit=100, date_start="2024-09-01")
df


,route_id,timepoint_stop_name,stop_order,direction,timepoint_stop_latitude_avg,timepoint_stop_longitude_avg
0,B1,86 ST/18 AV,14,S,40.6069830000000000,-74.0023840000000000
1,B1,86 ST/18 AV,38,N,40.6077240000000000,-74.0032600000000000
2,B1,86 ST/STILLWELL AV,22,S,40.5967320000000000,-73.9853610000000000
3,B1,86 ST/STILLWELL AV,30,N,40.5967910000000000,-73.9851950000000000
4,B1,87 ST/4 Av,1,S,40.6219240000000000,-74.0284950000000000
5,B1,AV X/MC DONALD AV,24,N,40.5901900000000000,-73.9740010000000000
6,B1,AV X/SHELL RD,27,S,40.5900990000000000,-73.9735220000000000
7,B1,BRIGHTON BEACH AV/BRIGHTON 1 PL,37,S,40.5766510000000000,-73.9654490000000000
8,B1,BRIGHTON BEACH AV/CONEY ISLAND AV,12,N,40.5779680000000000,-73.9602530000000000
9,B1,BRIGHTON BEACH/CONEY ISLAND AV,40,S,40.5779550000000000,-73.9596750000000000


,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence
0,B10025,40.578199,-73.939761,10001
1,B10025,40.578252,-73.939799,10002
2,B10025,40.578352,-73.939819,10003
3,B10025,40.578329,-73.940025,10004
4,B10025,40.578329,-73.940025,20001
...,...,...,...,...
165,B10025,40.595686,-73.983468,280003
166,B10025,40.595686,-73.983468,290001
167,B10025,40.595822,-73.983689,290002
168,B10025,40.596052,-73.984119,290003
